In [ ]:
# Imports
from pathlib import Path
import os
import rasterio
from rasterio.windows import Window
import PIL
import torchvision
import numpy
import cv2
import skimage
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
from albumentations import PadIfNeeded
import pandas as pd
from tqdm import tqdm
import shutil
from animaloc.data import ImageToPatches, PatchesBuffer, save_batch_images

# Splitting data for annotation

### For GeoTiffs

In [ ]:
def save_patches(img_path:Path,dest_dir:Path,tilesize=512):

    # create directory
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
        os.mkdir(dest_dir)
        print("emptying directory:", dest_dir)
    else:
        os.mkdir(dest_dir)
        print("creating directory:", dest_dir)
    
    # window reading with rasterio
    handler = rasterio.open(img_path)
    height, width = handler.meta['height'], handler.meta['width']
    coordinates = dict()
    count = 0
    for i,j in tqdm(product(list(range(0,height,tilesize)),list(range(0,width,tilesize)))):
        window = Window(j, i, tilesize, tilesize)
        
        try:
            chunk = handler.read(window=window)
            c,h,w = chunk.shape
            xmin, xmax = j, j+w
            ymin, ymax = i, i+h
            x_center = 0.5*(xmin+xmax)
            y_center = 0.5*(ymin+ymax)
            n_unique = np.unique(chunk).size
            if n_unique == 1:
                continue
            count += 1
            filename = img_path.name.split('.')[0] + f"-{j}-{i}.png"
            coordinates[count] = [xmin,xmax,ymin,ymax,x_center,y_center,filename]

            # save to disk
            chunk = np.transpose(chunk,(1,2,0))
            skimage.io.imsave(dest_dir/filename,chunk)          
                
        except Exception as e:
            print("Failed for",(i,j),e)
            pass

    cols = ['xmin','xmax','ymin','ymax','x_center','y_center','filename']
    coordinates = pd.DataFrame.from_dict(coordinates,
                                        orient='index',
                                        columns=cols)
    coordinates.to_csv(dest_dir/f"coordinates{img_path.name.split('.')[0]}.csv",index=False)
    handler.close()
    return coordinates

In [69]:
# for datapath in Path("../annotation_data/camp6/").iterdir():
datapath = Path("../annotation_data/camp6/150m_RGB.tif")
dest_dir = datapath.parent/(datapath.name.split('.')[0])
# coordinates =  save_patches(datapath,dest_dir,tilesize=1024)

emptying directory: ../annotation_data/camp6/150m_RGB


126it [01:09,  1.82it/s]


In [70]:
coordinates

,xmin,xmax,ymin,ymax,x_center,y_center,filename
1,0,1024,0,1024,512.0,512.0,150m_RGB-0-0.png
2,1024,2048,0,1024,1536.0,512.0,150m_RGB-1024-0.png
3,2048,3072,0,1024,2560.0,512.0,150m_RGB-2048-0.png
4,3072,4096,0,1024,3584.0,512.0,150m_RGB-3072-0.png
5,4096,5120,0,1024,4608.0,512.0,150m_RGB-4096-0.png
...,...,...,...,...,...,...,...
112,2048,3072,8192,8612,2560.0,8402.0,150m_RGB-2048-8192.png
113,3072,4096,8192,8612,3584.0,8402.0,150m_RGB-3072-8192.png
114,4096,5120,8192,8612,4608.0,8402.0,150m_RGB-4096-8192.png
115,5120,6144,8192,8612,5632.0,8402.0,150m_RGB-5120-8192.png


### For Images (jpg, rgb etc.)
Using Herdnet code

In [6]:
!pwd # current working dir

/home/ubuntu/workspace/HerdNet


In [11]:
# splitting validation data
Path("../general_dataset/val_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../general_dataset/val 640 640 100 \
    ../general_dataset/val_splits \
    -csv ../general_dataset/groundtruth/csv/val_big_size_A_B_E_K_WH_WB.csv \
    -min 0.0 -all False

Exporting patches: 100%|██████████████████████| 111/111 [06:00<00:00,  3.25s/it]


In [40]:
# splitting training data
Path("../general_dataset/train_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../general_dataset/train 640 640 100 \
    ../general_dataset/train_splits \
    -csv ../general_dataset/groundtruth/csv/train_big_size_A_B_E_K_WH_WB.csv \
    -min 0.0 -all False

Exporting patches:  27%|█████▉                | 248/928 [13:09<36:35,  3.23s/it]

In [ ]:
# splitting test data
Path("../general_dataset/test_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../general_dataset/test 640 640 100 \
    ../general_dataset/test_splits \
    -csv ../general_dataset/groundtruth/csv/test_big_size_A_B_E_K_WH_WB.csv \
    -min 0.0 -all False

In [8]:
# sample training data
# non empty
from pathlib import Path
import os
import random
directory = Path(r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\wildlife_localizer_data\train")
labels = directory/'labels'
dest_images = directory/'images_1e-1empty_nonempty'
source_images = directory/'images'

for file in labels.iterdir():
    img_name = file.name.split('.')[0]+'.JPG'
    if (source_images/img_name).exists():
        os.rename(src=source_images/img_name,
                dst=dest_images/img_name)
        
# and empty 
num_non_empty = len(list(labels.iterdir()))
num_empty_target = num_non_empty
empty_images = list(source_images.iterdir())
random.seed(41) # seeding for reproducibility
random.shuffle(empty_images) # shuffle
for file in empty_images[:num_empty_target]:
    img_name = file.name
    os.rename(src=file,
                dst=dest_images/img_name)

# Visualizing data

In [2]:
import albumentations as A
from animaloc.datasets import CSVDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [13]:
# patch_size = 640
# num_classes = 7
down_ratio = 1

val_dataset = CSVDataset(
    csv_file = '../wildlife_localizer_data/val/gt.csv',
    root_dir = '../wildlife_localizer_data/val',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='bbox')]
    )

In [18]:
val_dataset.anno_type

'BoundingBox'

In [14]:
labels = pd.read_csv('../wildlife_localizer_data/val/gt.csv')
labels.head()

,images,labels,base_images,x_min,y_min,x_max,y_max
0,005c952ba7a612c40986806cc84a87e1573ef4f2_13.JPG,6,005c952ba7a612c40986806cc84a87e1573ef4f2.JPG,243,255,283,307
1,005c952ba7a612c40986806cc84a87e1573ef4f2_15.JPG,6,005c952ba7a612c40986806cc84a87e1573ef4f2.JPG,399,254,452,302
2,005c952ba7a612c40986806cc84a87e1573ef4f2_23.JPG,6,005c952ba7a612c40986806cc84a87e1573ef4f2.JPG,293,283,339,308
3,031833f31b1622ec6701b7433a1664231f401d73_4.JPG,6,031833f31b1622ec6701b7433a1664231f401d73.JPG,183,144,226,181
4,031833f31b1622ec6701b7433a1664231f401d73_4.JPG,6,031833f31b1622ec6701b7433a1664231f401d73.JPG,118,275,157,295


In [12]:
np.sort(labels['labels'].unique())

array([1, 2, 3, 4, 5, 6])

In [24]:
# loader = DataLoader(dataset = val_dataset, batch_size = 1, shuffle = False)

In [4]:
images, targets = val_dataset[0]

In [5]:
targets

{'labels': tensor([6]),
 'base_images': ['005c952ba7a612c40986806cc84a87e1573ef4f2.JPG'],
 'boxes': tensor([[243., 255., 283., 307.]]),
 'image_id': [0],
 'image_name': ['005c952ba7a612c40986806cc84a87e1573ef4f2_13.JPG']}

In [39]:
# img = np.transpose(images.numpy(),(1,2,0))
# plt.imshow(img)

# Creating YOLO labels

In [13]:
# Images dimensions
height,width = 640, 640

# Create label directory
for directory in Path("../wildlife_localizer_data/").iterdir():
    if not directory.is_dir():
        continue 
    labels_dir = Path(os.path.join(directory,'labels'))
    labels_dir.mkdir(exist_ok=True,parents=False) # create directory if it does not exist
    labels = pd.read_csv(os.path.join(directory,'gt.csv'))
    #-- Saving labels in YOLO format
    for img_filename,df_group in tqdm(labels.groupby(by='images'),desc=directory.name):
        df_group['width'] = (df_group['x_max'] - df_group['x_min'])/width 
        df_group['height'] = (df_group['y_max'] - df_group['y_min'])/height
        df_group['x'] = (0.5*(df_group['x_min'] + df_group['x_max']))/width # x center
        df_group['y'] = (0.5*(df_group['y_min'] + df_group['y_max']))/height # y center
        df_group['labels'] = 0 

        # print('\n\n',directory.name,'\n',df_group[['labels','x','y','width','height']])
        # break

        # uncomment to save labels files
        labels_filename     = img_filename.split('.')[0] + '.txt'
        if len(df_group)>0:
            cols = ['labels','x','y','width','height']
            df_group[cols].to_csv(os.path.join(labels_dir,labels_filename),
                                    sep=" ",
                                    header=False,
                                    index=False)


test:   0%|          | 0/1633 [00:00<?, ?it/s]

train: 100%|██████████| 5186/5186 [00:22<00:00, 232.69it/s]


In [ ]:
# Control splitting
num_missing = 0
num_found = 0
num_total = 0
for path in Path("../wildlife_localizer_data/train/images").iterdir():
    filename = path.name.split('.')[0]
    labelpath = Path("../wildlife_localizer_data/train/labels")/(filename + '.txt')
    num_total += 1
    if not labelpath.exists():
        num_missing += 1
    else:
        num_found += 1

num_missing,num_found,num_total

In [7]:
labels = pd.read_csv(os.path.join(directory,'gt.csv'))
labels['labels'].unique()

array([6, 3, 2, 5, 4, 1])

# Training with YOLO
Using ultralytics library

In [3]:
from ultralytics import YOLO
import ultralytics

In [ ]:
# Creating a YOLO dataset using ultralytics > Debugging
import yaml
with open("../wildlife_localizer_data/data_config.yaml", 'r') as file:
    data_yaml = yaml.safe_load(file)
dataset = ultralytics.data.YOLODataset(data=data_yaml,
                                       img_path = "../wildlife_localizer_data/train/images",
                                       use_keypoints=False,
                                       single_cls=True,
                                       use_segments=False)
# view dataset
dataset[0]

In [4]:
# Load a model
model = YOLO("yolov8n.pt",task='detect')  # load a pretrained model (recommended for training)
model.info()

YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

In [5]:
!wandb offline

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


In [6]:
# Use the model
model.train(data="../wildlife_localizer_data/data_config.yaml",
            epochs=100,
            imgsz=640,
            batch=64,
            single_cls=True,
            patience=20,
            device=0,
            resume=False,
            dropout=0.1,
            iou=0.5,
            optimizer='Adam',
            pretrained=True,
            cos_lr=False,
            lr0=1e-3,
            lrf=1e-5)  # train the model


New https://pypi.org/project/ultralytics/8.0.209 available 😃 Update with 'pip install -U ultralytics'


Ultralytics YOLOv8.0.208 🚀 Python-3.11.5 torch-2.0.1+cu117 CPU (Intel Xeon Platinum 8175M 2.50GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=../wildlife_localizer_data/data_config.yaml, epochs=1, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torch

Freezing layer 'model.22.dfl.conv.weight'
train: Scanning /home/ubuntu/workspace/wildlife_localizer_data/train/labels... 5186 images, 72156 backgrounds, 0 corrupt: 100%|██████████| 77342/77342 [00:21<00:00, 3610.67it/s]
train: New cache created: /home/ubuntu/workspace/wildlife_localizer_data/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /home/ubuntu/workspace/wildlife_localizer_data/val/labels... 723 images, 8450 backgrounds, 0 corrupt: 100%|██████████| 9173/9173 [00:02<00:00, 3904.75it/s]
val: New cache created: /home/ubuntu/workspace/wildlife_localizer_data/val/labels.cache
Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 w

KeyboardInterrupt: 

In [ ]:
# metrics = model.val()  # evaluate model performance on the validation set
# results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
# path = model.export(format="onnx")  # export the model to ONNX format